# Translating between and combining the functional and the subclassing API

A lot of code that you will find online uses the functional API from tensorflow instead of the subclassing API. While translating the code between the APIs might seem difficult at first, it is rather straight forward. So before showing the architectural building blocks of the DenseNet, let's have a brief look at how to translate a simple model from the functional API to the subclassing API.

While we do not allow the use of the functional API in the homeworks, it may come in handy when working on your projects.

In [1]:
import tensorflow as tf

In [2]:
def composite_layer(x, dropout_rate):
    # instantiate a dense layer and directly call it on the input
    x = tf.keras.layers.Dense(32)(x)  
    x = tf.keras.layers.Activation(tf.nn.relu)(x)
    
    # add dropout
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    
    # do the same thing again
    x = tf.keras.layers.Dense(64)(x)
    x = tf.keras.layers.Activation(tf.nn.relu)(x)

    return x


def get_dense_model(dropout_rate):
    # we need to specify the input shape (ignoring the batch dimension)
    x_in = tf.keras.Input((16))
    
    # call the composite layer function on the input
    x = composite_layer(x_in, dropout_rate)
    
    # add an output layer
    x_out = tf.keras.layers.Dense(10, activation="softmax")(x)
    
    return tf.keras.Model(inputs=x_in, outputs=x_out)

functional_model = get_dense_model(dropout_rate=0.5)

functional_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16)]              0         
                                                                 
 dense (Dense)               (None, 32)                544       
                                                                 
 activation (Activation)     (None, 32)                0         
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 activation_1 (Activation)   (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 10)                650   

In [3]:
class CompositeLayer(tf.keras.layers.Layer):
    def __init__(self, dropout_rate):
        super(CompositeLayer, self).__init__()
        
        # instantiate layers in a list (useful if layers can be applied sequentially).
        self.layer_list = [
            
            tf.keras.layers.Dense(32),
            tf.keras.layers.Activation(tf.nn.relu),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(64),
            tf.keras.layers.Activation(tf.nn.relu)
            
        ]
        
    def call(self, x, training=False):
        
        # apply the layers from the list sequentially
        for layer in self.layer_list:
            
            # try to pass the training argument
            try:
                x = layer(x, training)
                
            # unless this does not work, then don't pass it
            except:
                x = layer(x)
        
        return x
    
    
    
class DenseModel(tf.keras.Model):
    def __init__(self, dropout_rate):
        super(DenseModel, self).__init__()
        
        self.composite_layer = CompositeLayer(dropout_rate)
        
        self.output_layer = tf.keras.layers.Dense(10, activation="softmax")
        
    def call(self, x, training=False):
        
        x = self.composite_layer(x, training)
        
        x = self.output_layer(x)
        
        return x

subclassed_model = DenseModel(dropout_rate=0.5)

# call the model on input to build the layers in it

subclassed_model(tf.ones((1,16)));

subclassed_model.summary()

Model: "dense_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 composite_layer (CompositeL  multiple                 2656      
 ayer)                                                           
                                                                 
 dense_5 (Dense)             multiple                  650       
                                                                 
Total params: 3,306
Trainable params: 3,306
Non-trainable params: 0
_________________________________________________________________


Finally, we can mix and match between the two APIs (same for the sequential API which we do not cover here).

So it is possible to write subclassed layers or modules and then use them within a functional API model. It is also possible to use functional API models as parts of a subclassed model or layer. The only difference that you will notice is that your layers will be encapsulated in models when you call model.layers or model.summary().

Something to keep in mind is that functional models are created for specific input dimensions because they build the layers while instantiating the model. Subclassed models on the other hand need to be built by calling them on an input.

In [9]:
# a functional API model that uses a subclassed layer

x_in = tf.keras.Input((16))
x = CompositeLayer(dropout_rate=0.5)(x_in)
x_out = tf.keras.layers.Dense(10, activation="softmax")(x)

mixed_model_a = tf.keras.Model(x_in, x_out)


# a subclassed model that uses a functional API model as part of it

class MixedModel(tf.keras.Model):
    def __init__(self):
        super(MixedModel, self).__init__()
        
        self.functional_model = get_dense_model(dropout_rate=0.5)
        
    def call(self, x, training=False):
        
        x = self.functional_model(x, training)
        
        return x
    
mixed_model_b = MixedModel()

mixed_model_b(tf.ones((1,16)))
mixed_model_b.summary()

Model: "mixed_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_5 (Functional)        (None, 10)                3306      
                                                                 
Total params: 3,306
Trainable params: 3,306
Non-trainable params: 0
_________________________________________________________________
